# Random Forest - baseline

* Referencing: 
> **Bibliography:** Zanella-Calzada, L.A., Galván-Tejada, C.E., Chávez-Lamas, N.M., Gracia-Cortés, M. del C., Magallanes-Quintanar, R., Celaya-Padilla, J.M., Galván-Tejada, J.I. and Gamboa-Rosales, H. (2019) Feature Extraction in Motor Activity Signal: Towards a Depression Episodes Detection in Unipolar and Bipolar Patients. Diagnostics [online]. 9 (1), p. 8. Available from: https://www.mdpi.com/2075-4418/9/1/8 [Accessed 28 November 2023].
* [article notes](../literature/Zanella-FeatureExtraction.md)

Objective: 

* To fit a modified Random Forest model, taking inspiration from the above article

## Plan 

1. Load and process `depresjon`
   * load into pandas df
   * select `control` and `condition` -> it seems that they used first 4 control and first 5 condition participants
   * normalise data (mean = 0, std = 1)
   * remove incomplete cases
   * take only first value of each hour??  maybe mean for each hour (**I don't understand this**)

2. Extract features - 14 features
   * mean
   * standard deviation
   * variance
   * trimmed mean
   * coefficient of variation
   * inversse coefficient of variation
   * kurtosis
   * skewness
   * quantailes (1, 5, 25, 75, 95, 99)

3. 

In [1]:
# libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, mstats
from statsmodels.robust import mad
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


## Load and preprocess data

### Load data files

In [2]:
def extract_folder(folderpath, add_scores=False, downsample=None):
    """
    Extract CSV data from folder and subfolders into a dataframe.

    Args:
      folderpath (str): Path to the folder containing CSV files.
      add_scores (bool, optional): Boolean to add scores.csv to the dataframe. Defaults to False.
      downsample (int, optional): Number of rows to downsample CSVs to. Defaults to None.

    Returns:
      pandas.DataFrame: DataFrame of concatenated CSV data.
    """

    # Dict to store dataframes by condition  
    dfs = {'control': [], 'condition': []}

    try:
        # Handle top-level scores CSV
        if add_scores and 'scores.csv' in os.listdir(folderpath):
            scores_path = os.path.join(folderpath, 'scores.csv')  
            dfs['scores'] = pd.read_csv(scores_path)

        # Get subfolders
        subfolders = [f for f in os.listdir(folderpath) if os.path.isdir(os.path.join(folderpath, f))]

        for subfolder in subfolders:
            subfolderpath = os.path.join(folderpath, subfolder)  

            # Get list of CSV files
            files = os.listdir(subfolderpath)

            for file in files:
                filepath = os.path.join(subfolderpath, file)

                # Extract ID from filename 
                id = file.split('.')[0]

                df = pd.read_csv(filepath)

                # Downsample if needed
                if downsample:
                    df = df.sample(downsample)

                # Add ID column - this is the filename without the extension
                df['id'] = id

                # Add 'condition' column
                df['condition'] = subfolder

                # Convert 'timestamp' and 'date' to datetime
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df['date'] = pd.to_datetime(df['date'])

                # Append to dict by condition
                if subfolder == 'control':
                    dfs['control'].append(df)
                else:  
                    dfs['condition'].append(df)

    except OSError:
        print(f"Error reading folder: {folderpath}")

    # concatenate dfs for each condition
    dfs['control'] = pd.concat(dfs['control'])
    dfs['condition'] = pd.concat(dfs['condition'])

    # Reset index on the final df
    df = pd.concat([dfs['control'], dfs['condition']]).reset_index(drop=True)

    # add label column
    df['label'] = 0
    df.loc[df['condition'] == 'condition', 'label'] = 1
    
    # remove old 'condition' column
    df.drop('condition', axis=1, inplace=True)

    # Final concat
    return df

In [3]:
# set folder path
folderpath = '../data/depresjon/'
# extract all files
all_files = extract_folder(folderpath)
# print rows 21-24
#print(all_files.iloc[21:25])
print(all_files.head(-5))


                  timestamp       date  activity           id  label
0       2003-03-18 15:00:00 2003-03-18        60    control_1      0
1       2003-03-18 15:01:00 2003-03-18         0    control_1      0
2       2003-03-18 15:02:00 2003-03-18       264    control_1      0
3       2003-03-18 15:03:00 2003-03-18       662    control_1      0
4       2003-03-18 15:04:00 2003-03-18       293    control_1      0
...                     ...        ...       ...          ...    ...
1571696 2004-06-10 14:58:00 2004-06-10         0  condition_9      1
1571697 2004-06-10 14:59:00 2004-06-10         0  condition_9      1
1571698 2004-06-10 15:00:00 2004-06-10         0  condition_9      1
1571699 2004-06-10 15:01:00 2004-06-10         5  condition_9      1
1571700 2004-06-10 15:02:00 2004-06-10         0  condition_9      1

[1571701 rows x 5 columns]


### Select subset

As per article - picking first 4 control & first 5 condition

In [4]:
control_subjects = ['control_1', 'control_2', 'control_3', 'control_4']
condition_subjects = ['condition_1', 'condition_2', 'condition_3', 'condition_4', 'condition_5']

# Filter for control subjects
control_df = all_files[all_files['id'].isin(control_subjects)] 

# Filter for condition subjects
condition_df = all_files[all_files['id'].isin(condition_subjects)]

# Concatenate 
subset_df = pd.concat([control_df, condition_df])

# print the first 5 rows
print(subset_df.head(5), '\n')

# print info
print(subset_df.info(), '\n')

# print random 10 rows
print(subset_df.sample(10), '\n')

# print number of rows by 'id'
print(subset_df['id'].value_counts(), '\n')

# print number of rows by 'label'
print(subset_df['label'].value_counts())

# print proportion of 'label' column
print(subset_df['label'].value_counts(normalize=True))

            timestamp       date  activity         id  label
0 2003-03-18 15:00:00 2003-03-18        60  control_1      0
1 2003-03-18 15:01:00 2003-03-18         0  control_1      0
2 2003-03-18 15:02:00 2003-03-18       264  control_1      0
3 2003-03-18 15:03:00 2003-03-18       662  control_1      0
4 2003-03-18 15:04:00 2003-03-18       293  control_1      0 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306813 entries, 0 to 1488480
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  306813 non-null  datetime64[ns]
 1   date       306813 non-null  datetime64[ns]
 2   activity   306813 non-null  int64         
 3   id         306813 non-null  object        
 4   label      306813 non-null  int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 14.0+ MB
None 

                  timestamp       date  activity           id  label
330295  2002-10-20 02:20:00 2002-10-20 

### Normalise subset (z-score)

Could use: 

* `sklearn.preprocessing.scale` - Standardises features by removing the mean and scaling to unit variance (similar to manual z-score normalisation)
* `sklearn.preprocessing.minmax_scale` - Transforms features to a given range (often 0-1 for minmax scaling)
* `sklearn.preprocessing.normalize` - L2 vector normalisation

* pandas -> df.normalize(axis=0)

In [5]:
# calculate mean and standard deviation
mu = subset_df['activity'].mean()
sigma = subset_df['activity'].std()

# normalise
normal_df = subset_df.copy()
normal_df['activity_norm'] = (normal_df['activity'] - mu)/sigma

# print summary statistics
print(normal_df.describe())

            activity          label  activity_norm
count  306813.000000  306813.000000   3.068130e+05
mean      168.451369       0.413499  -2.501153e-17
std       356.899613       0.492462   1.000000e+00
min         0.000000       0.000000  -4.719853e-01
25%         0.000000       0.000000  -4.719853e-01
50%         3.000000       0.000000  -4.635796e-01
75%       160.000000       1.000000  -2.367996e-02
max      6776.000000       1.000000   1.851375e+01


### Missing values

In [6]:
# Check if dataframe has any NaN 
print(normal_df.isnull().values.any(), '\n')

# Count number of NaN per column
print(normal_df.isnull().sum(), '\n')

# See indices of NaN values 
print(normal_df[normal_df.isnull().any(axis=1)].index)

False 

timestamp        0
date             0
activity         0
id               0
label            0
activity_norm    0
dtype: int64 

Int64Index([], dtype='int64')


In [7]:
# drop NaN values rows
normal_df.dropna(inplace=True)

# Check if dataframe has any NaN
print(normal_df.isnull().values.any())

# print info
print(normal_df.info())

False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 306813 entries, 0 to 1488480
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   timestamp      306813 non-null  datetime64[ns]
 1   date           306813 non-null  datetime64[ns]
 2   activity       306813 non-null  int64         
 3   id             306813 non-null  object        
 4   label          306813 non-null  int64         
 5   activity_norm  306813 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 16.4+ MB
None


## Extract features (14)





**Mean**:

$\mu = \frac{1}{n} \sum_{i=1}^{n} x_i$

* Calculation: Average value of the activity data.
* Significance: Represents the central tendency of the data.

**Trimmed Standard Deviation:**

$\text{sd} = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} (x_i - \mu)^2}$


* Calculation: A measure of the amount of variation in the data, with a specified percentage of outliers removed.
* Significance: More robust to outliers than the regular standard deviation.

**Trimmed Variance:**

$\text{sd}^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2$

* Calculation: Similar to trimmed standard deviation but squared, providing a measure of the spread of the data.
* Significance: Robust measure of data spread with specified outliers removed.

**Quantiles (1st, 5th, 25th, 75th, 95th, 99th):**

$Q[i](p) = (1 - \gamma)x[j] + \gamma x[j + 1]$

* Calculation: Values below which a given percentage of the data falls.
* Significance: Describes the distribution of the data and helps identify potential outliers.

**Skewness:**

$S = \frac{\mu - \upsilon}{\sigma}$

* Calculation: A measure of the asymmetry of the data distribution.
* Significance: Positive skewness indicates a right-skewed distribution (tail to the right), negative skewness indicates a left-skewed distribution (tail to the left).

**Kurtosis:**

$K = \frac{\mu}{\sigma}$

* Calculation: A measure of the "tailedness" of the data distribution.
* Significance: High kurtosis indicates heavy tails and more outliers compared to a normal distribution. Low kurtosis indicates light tails.

**Coefficient of Variation (Coef_Var):**

$\text{CV} = \frac{\text{sd}}{\mu}$

* Calculation: Standard deviation divided by the mean, expressing the relative variability of the data.
* Significance: Useful for comparing the variability of datasets with different units or scales.

**Inverse Coefficient of Variation (Inverse_Coef_Var):**

$\text{ICV} = \frac{\mu}{\text{sd}}$

* Calculation: The reciprocal of the coefficient of variation.
* Significance: Measures the efficiency of data representation; a higher value suggests more efficient data representation.

In [9]:
# Copy df to avoid changing the original
normal_feature_df = normal_df.copy()

# 'timestamp' as index
normal_feature_df = normal_feature_df.set_index('timestamp')
normal_df = normal_df.set_index('timestamp')

In [11]:
# Mean, standard deviation, and variance for each hour
mean_value_hourly = normal_feature_df.resample('H')['activity_norm'].mean()
std_value_hourly = normal_feature_df.resample('H')['activity_norm'].std()
variance_value_hourly = normal_feature_df.resample('H')['activity_norm'].var()


In [12]:
# Trimmed Mean (using 5% trimming??) for each hour
def calculate_trimmed_mean(x):
    if len(x) > 0:
        return np.mean(x[(x >= np.percentile(x, 5)) & (x <= np.percentile(x, 95))])
    else:
        return np.nan

trimmed_mean_value_hourly = normal_feature_df.resample('H')['activity_norm'].apply(calculate_trimmed_mean)

# Coefficient of Variation and Inverse Coefficient of Variation for each hour
coef_var_value_hourly = std_value_hourly / mean_value_hourly
inverse_coef_var_value_hourly = mean_value_hourly / std_value_hourly


In [13]:
# Kurtosis and Skewness for each hour
def calculate_kurtosis(x):
    if len(x) > 0 and np.std(x) != 0:
        return kurtosis(x)
    else:
        return np.nan

def calculate_skewness(x):
    if len(x) > 0 and np.std(x) != 0:
        return skew(x)
    else:
        return np.nan

# Kurtosis and Skewness for each hour
kurtosis_value_hourly = normal_feature_df.resample('H')['activity_norm'].apply(lambda x: kurtosis(x) if len(x) > 0 else np.nan)
skewness_value_hourly = normal_feature_df.resample('H')['activity_norm'].apply(lambda x: skew(x) if len(x) > 0 else np.nan)



C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\514108838.py:15: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_value_hourly = normal_feature_df.resample('H')['activity_norm'].apply(lambda x: kurtosis(x) if len(x) > 0 else np.nan)
C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\514108838.py:16: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness_value_hourly = normal_feature_df.resample('H')['activity_norm'].apply(lambda x: skew(x) if len(x) > 0 else np.nan)


In [14]:
# Quantiles (1%, 5%, 25%, 75%, 95%, 99%) for each hour
quantiles_value_hourly = normal_feature_df.resample('H')['activity_norm'].quantile([0.01, 0.05, 0.25, 0.75, 0.95, 0.99])


In [15]:
# Extract 'label' from normal_df with resampling
label_hourly = normal_df.resample('H')['label'].first()

In [16]:
# Align labels with features
features_hourly = pd.DataFrame({
    'mean': mean_value_hourly,
    'std': std_value_hourly,
    'variance': variance_value_hourly,
    'trimmed_mean': trimmed_mean_value_hourly,
    'coef_var': coef_var_value_hourly,
    'inverse_coef_var': inverse_coef_var_value_hourly,
    'kurtosis': kurtosis_value_hourly,
    'skewness': skewness_value_hourly,
    'label': label_hourly  # Add 'label' column
})

In [17]:
# Add Quantiles
features_hourly = pd.concat([features_hourly, quantiles_value_hourly.unstack(level=-1)], axis=1)

In [18]:
print(features_hourly)

                         mean       std  variance  trimmed_mean   coef_var  \
timestamp                                                                    
2002-10-02 15:00:00  1.226905  1.213061  1.471516      1.122097   0.988716   
2002-10-02 16:00:00  1.574949  1.221473  1.491997      1.543573   0.775564   
2002-10-02 17:00:00  2.149760  1.191689  1.420123      2.106185   0.554336   
2002-10-02 18:00:00  0.310868  1.101726  1.213801      0.145472   3.544034   
2002-10-02 19:00:00  0.318620  1.047779  1.097842      0.157313   3.288494   
...                       ...       ...       ...           ...        ...   
2003-06-27 04:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 05:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 06:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 07:00:00 -0.043854  0.710295  0.504518     -0.170510 -16.196911   
2003-06-27 08:00:00 -0.433736  0.184898  0.034187     -0.471985 

In [19]:
# Check for NaN or inf values
nan_inf_check = features_hourly.isnull().sum().append(features_hourly.isin([np.inf, -np.inf]).sum())
print("NaN and Inf Check:")
print(nan_inf_check)


NaN and Inf Check:
mean                2488
std                 2488
variance            2488
trimmed_mean        2488
coef_var            2488
inverse_coef_var    2488
kurtosis            3665
skewness            3665
label               2488
0.01                2488
0.05                2488
0.25                2488
0.75                2488
0.95                2488
0.99                2488
mean                   0
std                    0
variance               0
trimmed_mean           0
coef_var               0
inverse_coef_var    1177
kurtosis               0
skewness               0
label                  0
0.01                   0
0.05                   0
0.25                   0
0.75                   0
0.95                   0
0.99                   0
dtype: int64


C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\2716595254.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_inf_check = features_hourly.isnull().sum().append(features_hourly.isin([np.inf, -np.inf]).sum())


In [20]:
# drop rows with missing labels
features_hourly_2 = features_hourly.dropna(subset=['label'])

#print shape
print(features_hourly_2.shape)

nan_inf_check = features_hourly_2.isnull().sum().append(features_hourly_2.isin([np.inf, -np.inf]).sum())
print("NaN and Inf Check:")
print(nan_inf_check)

(3938, 15)
NaN and Inf Check:
mean                   0
std                    0
variance               0
trimmed_mean           0
coef_var               0
inverse_coef_var       0
kurtosis            1177
skewness            1177
label                  0
0.01                   0
0.05                   0
0.25                   0
0.75                   0
0.95                   0
0.99                   0
mean                   0
std                    0
variance               0
trimmed_mean           0
coef_var               0
inverse_coef_var    1177
kurtosis               0
skewness               0
label                  0
0.01                   0
0.05                   0
0.25                   0
0.75                   0
0.95                   0
0.99                   0
dtype: int64


C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\198874199.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_inf_check = features_hourly_2.isnull().sum().append(features_hourly_2.isin([np.inf, -np.inf]).sum())


Rows with NaN kurtosis and skewness and -inf inverse_coef_var...

**Decided to impute to mean for now.**

In [21]:
# Impute missing values with mean or median
features_hourly_2['kurtosis'].fillna(features_hourly_2['kurtosis'].mean(), inplace=True)
features_hourly_2['skewness'].fillna(features_hourly_2['skewness'].mean(), inplace=True)

# Handle infinite values in 'inverse_coef_var'
features_hourly_2.replace([np.inf, -np.inf], np.nan, inplace=True)
features_hourly_2['inverse_coef_var'].fillna(features_hourly_2['inverse_coef_var'].mean(), inplace=True)


C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\1054803192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_hourly_2['kurtosis'].fillna(features_hourly_2['kurtosis'].mean(), inplace=True)
C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\1054803192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_hourly_2['skewness'].fillna(features_hourly_2['skewness'].mean(), inplace=True)
C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\1054803192.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [22]:
nan_inf_check = features_hourly_2.isnull().sum().append(features_hourly_2.isin([np.inf, -np.inf]).sum())
print("NaN and Inf Check:")
print(nan_inf_check)

NaN and Inf Check:
mean                0
std                 0
variance            0
trimmed_mean        0
coef_var            0
inverse_coef_var    0
kurtosis            0
skewness            0
label               0
0.01                0
0.05                0
0.25                0
0.75                0
0.95                0
0.99                0
mean                0
std                 0
variance            0
trimmed_mean        0
coef_var            0
inverse_coef_var    0
kurtosis            0
skewness            0
label               0
0.01                0
0.05                0
0.25                0
0.75                0
0.95                0
0.99                0
dtype: int64


C:\Users\zoona\AppData\Local\Temp\ipykernel_32272\1165047148.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan_inf_check = features_hourly_2.isnull().sum().append(features_hourly_2.isin([np.inf, -np.inf]).sum())


## Modelling

### Train / Test split

* test = 0.2
* random seed

In [23]:
"""
Split the data into training and testing sets.

Parameters:
    X : pandas.DataFrame
        The feature matrix.
    y : pandas.Series
        The target variable.
    test_size : float, optional
        The proportion of the dataset to include in the test split. Default is 0.2.
    random_state : int, optional
        The seed used by the random number generator. Default is 42.

Returns:
    X_train : pandas.DataFrame
        The training feature matrix.
    X_test : pandas.DataFrame
        The testing feature matrix.
    y_train : pandas.Series
        The training target variable.
    y_test : pandas.Series
        The testing target variable.
"""
X = features_hourly_2.drop('label', axis=1)
y = features_hourly_2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Model selection - Random Forest

In [24]:
# model
rf_model = RandomForestClassifier(random_state=42)


### Hyperparameters

* estimators - number of trees in the forest; how many trees before deciding
* max_depth - max depth of tree - levels
* min_samples_split - min number of samples required to split an internal node
* min_samples_leaf - min number of samples at leaf node

Trains best model using grid search to find best hyperparameters.

In [25]:
from sklearn.model_selection import GridSearchCV

# Convert column names to strings
X_train.columns = X_train.columns.astype(str)

# hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# grid search
grid_search = GridSearchCV(rf_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_


### Train model

Train on training set

In [26]:
best_rf_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=200,
                       random_state=42)

### Predictions and Evaluation

In [27]:
# Convert column names to strings for X_test
X_test.columns = X_test.columns.astype(str)

y_pred = best_rf_model.predict(X_test)


In [28]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report)
print("Confusion Matrix:\n", confusion_mat)


Accuracy: 0.8388324873096447
Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.91      0.89       554
         1.0       0.76      0.67      0.71       234

    accuracy                           0.84       788
   macro avg       0.81      0.79      0.80       788
weighted avg       0.83      0.84      0.84       788

Confusion Matrix:
 [[505  49]
 [ 78 156]]


In [29]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(best_rf_model, X_train, y_train, cv=5)
mean_cv_score = np.mean(cv_scores)


In [31]:
# print cross-validation scores
print(cv_scores)

# print mean cross-validation score
print(mean_cv_score)

[0.82063492 0.80793651 0.84285714 0.81587302 0.81428571]
0.8203174603174602


## SCRAPS - to be deleted

In [ ]:
# 'timestamp' as index
normal_df = normal_df.set_index('timestamp')

# extract 'label' from normal_df
label_hourly = normal_df.resample('H')['label'].first()

# merge features_hourly with label_hourly
merged_hourly_df = pd.merge(features_hourly, label_hourly, left_index=True, right_index=True)

# Print
print(merged_hourly_df)


                         mean       std  variance  trimmed_mean   coef_var  \
timestamp                                                                    
2002-10-02 15:00:00  1.226905  1.213061  1.471516      1.122097   0.988716   
2002-10-02 16:00:00  1.574949  1.221473  1.491997      1.543573   0.775564   
2002-10-02 17:00:00  2.149760  1.191689  1.420123      2.106185   0.554336   
2002-10-02 18:00:00  0.310868  1.101726  1.213801      0.145472   3.544034   
2002-10-02 19:00:00  0.318620  1.047779  1.097842      0.157313   3.288494   
...                       ...       ...       ...           ...        ...   
2003-06-27 04:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 05:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 06:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 07:00:00 -0.043854  0.710295  0.504518     -0.170510 -16.196911   
2003-06-27 08:00:00 -0.433736  0.184898  0.034187     -0.471985 

In [ ]:
# Extract 'label' from normal_df with resampling
label_hourly = normal_df.resample('H')['label'].first()

# Align labels with features
merged_hourly_df['label'] = label_hourly.loc[merged_hourly_df.index]

# Drop rows with NaN labels
merged_hourly_df = merged_hourly_df.dropna(subset=['label'])


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from scipy.stats import mstats

# Copy df to avoid changing the original
normmal_feature_df = normal_df.copy()

# 'timestamp' as index
normmal_feature_df = normmal_feature_df.set_index('timestamp')
normal_df = normal_df.set_index('timestamp')

# Mean, standard deviation, and variance for each hour
mean_value_hourly = normmal_feature_df.resample('H')['activity_norm'].mean()
std_value_hourly = normmal_feature_df.resample('H')['activity_norm'].std()
variance_value_hourly = normmal_feature_df.resample('H')['activity_norm'].var()

# Trimmed Mean (using 5% trimming??) for each hour
def calculate_trimmed_mean(x):
    if len(x) > 0:
        return np.mean(x[(x >= np.percentile(x, 5)) & (x <= np.percentile(x, 95))])
    else:
        return np.nan

trimmed_mean_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(calculate_trimmed_mean)

# Coefficient of Variation and Inverse Coefficient of Variation for each hour
coef_var_value_hourly = std_value_hourly / mean_value_hourly
inverse_coef_var_value_hourly = mean_value_hourly / std_value_hourly

# Kurtosis and Skewness for each hour
kurtosis_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: kurtosis(x) if len(x) > 0 else np.nan)
skewness_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: skew(x) if len(x) > 0 else np.nan)

# Quantiles (1%, 5%, 25%, 75%, 95%, 99%) for each hour
quantiles_value_hourly = normmal_feature_df.resample('H')['activity_norm'].quantile([0.01, 0.05, 0.25, 0.75, 0.95, 0.99])

# Extract 'label' from normal_df with resampling
label_hourly = normal_df.resample('H')['label'].first()

# Align labels with features
features_hourly = pd.DataFrame({
    'mean': mean_value_hourly,
    'std': std_value_hourly,
    'variance': variance_value_hourly,
    'trimmed_mean': trimmed_mean_value_hourly,
    'coef_var': coef_var_value_hourly,
    'inverse_coef_var': inverse_coef_var_value_hourly,
    'kurtosis': kurtosis_value_hourly,
    'skewness': skewness_value_hourly,
    'label': label_hourly  # Add 'label' column
})

# Add Quantiles
features_hourly = pd.concat([features_hourly, quantiles_value_hourly.unstack(level=-1)], axis=1)

# Rename columns
features_hourly.columns = ['mean', 'std', 'variance', 'trimmed_mean', 'coef_var', 'inverse_coef_var', 'kurtosis', 'skewness', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99', 'label']

# Print
print(features_hourly)


C:\Users\zoona\AppData\Local\Temp\ipykernel_12084\3898015565.py:32: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: kurtosis(x) if len(x) > 0 else np.nan)
C:\Users\zoona\AppData\Local\Temp\ipykernel_12084\3898015565.py:33: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: skew(x) if len(x) > 0 else np.nan)


                         mean       std  variance  trimmed_mean   coef_var  \
timestamp                                                                    
2002-10-02 15:00:00  1.226905  1.213061  1.471516      1.122097   0.988716   
2002-10-02 16:00:00  1.574949  1.221473  1.491997      1.543573   0.775564   
2002-10-02 17:00:00  2.149760  1.191689  1.420123      2.106185   0.554336   
2002-10-02 18:00:00  0.310868  1.101726  1.213801      0.145472   3.544034   
2002-10-02 19:00:00  0.318620  1.047779  1.097842      0.157313   3.288494   
...                       ...       ...       ...           ...        ...   
2003-06-27 04:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 05:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 06:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 07:00:00 -0.043854  0.710295  0.504518     -0.170510 -16.196911   
2003-06-27 08:00:00 -0.433736  0.184898  0.034187     -0.471985 

In [ ]:
# copy df to avoid changing the original
normmal_feature_df = normal_df.copy()


# 'timestamp' as index
normmal_feature_df = normmal_feature_df.set_index('timestamp')

# mean, standard deviation, and variance for each hour
mean_value_hourly = normmal_feature_df.resample('H')['activity_norm'].mean()
std_value_hourly = normmal_feature_df.resample('H')['activity_norm'].std()
variance_value_hourly = normmal_feature_df.resample('H')['activity_norm'].var()

# Trimmed Mean (using 5% trimming??) for each hour
def calculate_trimmed_mean(x):
    if len(x) > 0:
        return np.mean(x[(x >= np.percentile(x, 5)) & (x <= np.percentile(x, 95))])
    else:
        return np.nan

trimmed_mean_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(calculate_trimmed_mean)

# Coefficient of Variation and Inverse Coefficient of Variation for each hour
coef_var_value_hourly = std_value_hourly / mean_value_hourly
inverse_coef_var_value_hourly = mean_value_hourly / std_value_hourly

#  Kurtosis and Skewness for each hour
#kurtosis_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: kurtosis(x) if len(x) > 0 else np.nan)
#skewness_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: skew(x) if len(x) > 0 else np.nan)
kurtosis_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: mstats.kurtosis(x))
skewness_value_hourly = normmal_feature_df.resample('H')['activity_norm'].apply(lambda x: mstats.skew(x))

# Quantiles (1%, 5%, 25%, 75%, 95%, 99%) for each hour
quantiles_value_hourly = normmal_feature_df.resample('H')['activity_norm'].quantile([0.01, 0.05, 0.25, 0.75, 0.95, 0.99])

# df to store the features
features_hourly = pd.DataFrame({
    'mean': mean_value_hourly,
    'std': std_value_hourly,
    'variance': variance_value_hourly,
    'trimmed_mean': trimmed_mean_value_hourly,
    'coef_var': coef_var_value_hourly,
    'inverse_coef_var': inverse_coef_var_value_hourly,
    'kurtosis': kurtosis_value_hourly,
    'skewness': skewness_value_hourly
})

# Add Quantiles
features_hourly = pd.concat([features_hourly, quantiles_value_hourly.unstack(level=-1)], axis=1)

# Rename columns
features_hourly.columns = ['mean', 'std', 'variance', 'trimmed_mean', 'coef_var', 'inverse_coef_var', 'kurtosis', 'skewness', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99']

# Print
print(features_hourly)


c:\Users\zoona\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\zoona\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


                         mean       std  variance  trimmed_mean   coef_var  \
timestamp                                                                    
2002-10-02 15:00:00  1.226905  1.213061  1.471516      1.122097   0.988716   
2002-10-02 16:00:00  1.574949  1.221473  1.491997      1.543573   0.775564   
2002-10-02 17:00:00  2.149760  1.191689  1.420123      2.106185   0.554336   
2002-10-02 18:00:00  0.310868  1.101726  1.213801      0.145472   3.544034   
2002-10-02 19:00:00  0.318620  1.047779  1.097842      0.157313   3.288494   
...                       ...       ...       ...           ...        ...   
2003-06-27 04:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 05:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 06:00:00 -0.471985  0.000000  0.000000     -0.471985  -0.000000   
2003-06-27 07:00:00 -0.043854  0.710295  0.504518     -0.170510 -16.196911   
2003-06-27 08:00:00 -0.433736  0.184898  0.034187     -0.471985 

In [ ]:

# Check the index type and format for features_hourly
print("Features dataframe:")
print(features_hourly.index)
print(features_hourly.index.dtype)

# Check the index type and format for normal_df
print("\nLabel dataframe:")
print(normal_df.index)
print(normal_df.index.dtype)


Features dataframe:
DatetimeIndex(['2002-10-02 15:00:00', '2002-10-02 16:00:00',
               '2002-10-02 17:00:00', '2002-10-02 18:00:00',
               '2002-10-02 19:00:00', '2002-10-02 20:00:00',
               '2002-10-02 21:00:00', '2002-10-02 22:00:00',
               '2002-10-02 23:00:00', '2002-10-03 00:00:00',
               ...
               '2003-06-26 23:00:00', '2003-06-27 00:00:00',
               '2003-06-27 01:00:00', '2003-06-27 02:00:00',
               '2003-06-27 03:00:00', '2003-06-27 04:00:00',
               '2003-06-27 05:00:00', '2003-06-27 06:00:00',
               '2003-06-27 07:00:00', '2003-06-27 08:00:00'],
              dtype='datetime64[ns]', name='timestamp', length=6426, freq='H')
datetime64[ns]

Label dataframe:
DatetimeIndex(['2003-03-18 15:00:00', '2003-03-18 15:01:00',
               '2003-03-18 15:02:00', '2003-03-18 15:03:00',
               '2003-03-18 15:04:00', '2003-03-18 15:05:00',
               '2003-03-18 15:06:00', '2003-03-18 15:07

### v1 first hour

In [ ]:
# Groupby to calculate other features
features_first_hour = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].agg([np.mean,
                                lambda x: mstats.trimmed_std(x, 0.1),  # Adjust the trim parameter as needed
                                lambda x: mstats.trimmed_var(x, 0.1),
                                lambda x: np.quantile(x, 0.01), 
                                lambda x: np.quantile(x, 0.05),
                                lambda x: np.quantile(x, 0.25),
                                lambda x: np.quantile(x, 0.75),
                                lambda x: np.quantile(x, 0.95),
                                lambda x: np.quantile(x, 0.99)]) 

features_first_hour.columns = ['mean', 'std', 'variance', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99']

# Calculate skewness and kurtosis separately
skewness_values = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(mstats.skew)
kurtosis_values = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(mstats.kurtosis)

# Assign skewness and kurtosis to the dataframe
features_first_hour['skewness'] = skewness_values
features_first_hour['kurtosis'] = kurtosis_values

# Calculate remaining features
features_first_hour['coef_var'] = features_first_hour['std'] / features_first_hour['mean'] 
features_first_hour['inverse_coef_var'] = features_first_hour['mean'] / (np.abs(features_first_hour['std']) + 1e-9)


# print the first 5 rows
print(features_first_hour.head(5), '\n')


       mean  std  variance  quantile_01  quantile_05  quantile_25  \
0 -0.484596  0.0       0.0    -0.484596    -0.484596    -0.484596   
1  0.354670  0.0       0.0     0.354670     0.354670     0.354670   
2 -0.484596  0.0       0.0    -0.484596    -0.484596    -0.484596   
3 -0.380374  0.0       0.0    -0.380374    -0.380374    -0.380374   
4 -0.443456  0.0       0.0    -0.443456    -0.443456    -0.443456   

   quantile_75  quantile_95  quantile_99 skewness  kurtosis  coef_var  \
0    -0.484596    -0.484596    -0.484596      0.0      -3.0      -0.0   
1     0.354670     0.354670     0.354670      0.0      -3.0       0.0   
2    -0.484596    -0.484596    -0.484596      0.0      -3.0      -0.0   
3    -0.380374    -0.380374    -0.380374      0.0      -3.0      -0.0   
4    -0.443456    -0.443456    -0.443456      0.0      -3.0      -0.0   

   inverse_coef_var  
0     -4.845962e+08  
1      3.546704e+08  
2     -4.845962e+08  
3     -3.803736e+08  
4     -4.434557e+08   



### v2 mean hour

In [ ]:
# Groupby to calculate other features
features_mean_hour = subset_mean_hour_df.groupby(subset_mean_hour_df.index)['activity_norm'].agg([np.mean,
                                lambda x: mstats.trimmed_std(x, 0.1),  # Adjust the trim parameter as needed
                                lambda x: mstats.trimmed_var(x, 0.1),
                                lambda x: np.quantile(x, 0.01), 
                                lambda x: np.quantile(x, 0.05),
                                lambda x: np.quantile(x, 0.25),
                                lambda x: np.quantile(x, 0.75),
                                lambda x: np.quantile(x, 0.95),
                                lambda x: np.quantile(x, 0.99)]) 

features_mean_hour.columns = ['mean', 'std', 'variance', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99']

# skewness and kurtosis 
skewness_values = subset_mean_hour_df.groupby(subset_mean_hour_df.index)['activity_norm'].apply(mstats.skew)
kurtosis_values = subset_mean_hour_df.groupby(subset_mean_hour_df.index)['activity_norm'].apply(mstats.kurtosis)

# assign skewness and kurtosis to the dataframe
features_mean_hour['skewness'] = skewness_values
features_mean_hour['kurtosis'] = kurtosis_values

# coef and inverse_coef features
features_mean_hour['coef_var'] = features_mean_hour['std'] / features_mean_hour['mean'] 
features_mean_hour['inverse_coef_var'] = features_mean_hour['mean'] / (np.abs(features_mean_hour['std']) + 1e-9)


# print the first 5 rows
print(features_mean_hour.head(5), '\n')


       mean  std  variance  quantile_01  quantile_05  quantile_25  \
0  0.696000  0.0       0.0     0.696000     0.696000     0.696000   
1  0.453893  0.0       0.0     0.453893     0.453893     0.453893   
2  0.432865  0.0       0.0     0.432865     0.432865     0.432865   
3  0.196942  0.0       0.0     0.196942     0.196942     0.196942   
4  0.274151  0.0       0.0     0.274151     0.274151     0.274151   

   quantile_75  quantile_95  quantile_99 skewness  kurtosis  coef_var  \
0     0.696000     0.696000     0.696000      0.0      -3.0       0.0   
1     0.453893     0.453893     0.453893      0.0      -3.0       0.0   
2     0.432865     0.432865     0.432865      0.0      -3.0       0.0   
3     0.196942     0.196942     0.196942      0.0      -3.0       0.0   
4     0.274151     0.274151     0.274151      0.0      -3.0       0.0   

   inverse_coef_var  
0      6.960005e+08  
1      4.538928e+08  
2      4.328654e+08  
3      1.969423e+08  
4      2.741511e+08   



In [ ]:
print(features_first_hour.shape)
print(features_mean_hour.shape)

(5120, 13)
(5120, 13)


In [ ]:
features_first_hour = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].agg([np.mean, np.std, np.var, 
                                lambda x: np.quantile(x, 0.01), 
                                lambda x: np.quantile(x, 0.05),
                                lambda x: np.quantile(x, 0.25),
                                lambda x: np.quantile(x, 0.75),
                                lambda x: np.quantile(x, 0.95),
                                lambda x: np.quantile(x, 0.99),
                                skew, kurtosis]) 
features_first_hour.columns = ['mean', 'std', 'variance', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99','skewness', 'kurtosis']

features_first_hour['coef_var'] = features_first_hour['std'] / features_first_hour['mean'] 
features_first_hour['inverse_coef_var'] = features_first_hour['mean'] / features_first_hour['std']

# print the first 5 rows
print(features_first_hour.head(5), '\n')

c:\Users\zoona\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\groupby\groupby.py:1649: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f = lambda x: func(x, *args, **kwargs)


       mean  std  variance  quantile_01  quantile_05  quantile_25  \
0 -0.484596  NaN       NaN    -0.484596    -0.484596    -0.484596   
1  0.354670  NaN       NaN     0.354670     0.354670     0.354670   
2 -0.484596  NaN       NaN    -0.484596    -0.484596    -0.484596   
3 -0.380374  NaN       NaN    -0.380374    -0.380374    -0.380374   
4 -0.443456  NaN       NaN    -0.443456    -0.443456    -0.443456   

   quantile_75  quantile_95  quantile_99  skewness  kurtosis  coef_var  \
0    -0.484596    -0.484596    -0.484596       NaN       NaN       NaN   
1     0.354670     0.354670     0.354670       NaN       NaN       NaN   
2    -0.484596    -0.484596    -0.484596       NaN       NaN       NaN   
3    -0.380374    -0.380374    -0.380374       NaN       NaN       NaN   
4    -0.443456    -0.443456    -0.443456       NaN       NaN       NaN   

   inverse_coef_var  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN   



### v1 first hour

In [97]:




# Calculate Mean
mean_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].mean()

# Calculate Standard Deviation
std_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].std()

# Calculate Variance
variance_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].var()

# Calculate Trimmed Mean (using 5% trimming as an example)
trimmed_mean_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(lambda x: np.mean(x[(x >= np.percentile(x, 5)) & (x <= np.percentile(x, 95))]))

# Calculate Coefficient of Variation
coef_var_value = std_value / mean_value

# Calculate Inverse Coefficient of Variation
inverse_coef_var_value = mean_value / std_value

# Calculate Kurtosis
kurtosis_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(lambda x: kurtosis(x))

# Calculate Skewness
skewness_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(lambda x: skew(x))

# Calculate Quantiles (1%, 5%, 25%, 75%, 95%, 99%)
quantiles_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].quantile([0.01, 0.05, 0.25, 0.75, 0.95, 0.99]).unstack(level=-1)

# Create a dataframe to store the features
features_first_hour = pd.DataFrame({
    'mean': mean_value,
    'std': std_value,
    'variance': variance_value,
    'trimmed_mean': trimmed_mean_value,
    'coef_var': coef_var_value,
    'inverse_coef_var': inverse_coef_var_value,
    'kurtosis': kurtosis_value,
    'skewness': skewness_value
})

# Add Quantiles to the dataframe
features_first_hour = pd.concat([features_first_hour, quantiles_value], axis=1)

# Rename columns for clarity
features_first_hour.columns = ['mean', 'std', 'variance', 'trimmed_mean', 'coef_var', 'inverse_coef_var', 'kurtosis', 'skewness', 'quantile_01', 'quantile_05', 'quantile_25', 'quantile_75', 'quantile_95', 'quantile_99']

# Print or further use features_hour
print(features_first_hour)


C:\Users\zoona\AppData\Local\Temp\ipykernel_44496\3856034703.py:20: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(lambda x: kurtosis(x))
C:\Users\zoona\AppData\Local\Temp\ipykernel_44496\3856034703.py:23: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness_value = subset_first_hour_df.groupby(subset_first_hour_df.index)['activity_norm'].apply(lambda x: skew(x))


          mean  std  variance  trimmed_mean  coef_var  inverse_coef_var  \
0    -0.484596  NaN       NaN     -0.484596       NaN               NaN   
1     0.354670  NaN       NaN      0.354670       NaN               NaN   
2    -0.484596  NaN       NaN     -0.484596       NaN               NaN   
3    -0.380374  NaN       NaN     -0.380374       NaN               NaN   
4    -0.443456  NaN       NaN     -0.443456       NaN               NaN   
...        ...  ...       ...           ...       ...               ...   
5115 -0.470883  NaN       NaN     -0.470883       NaN               NaN   
5116 -0.193870  NaN       NaN     -0.193870       NaN               NaN   
5117 -0.476368  NaN       NaN     -0.476368       NaN               NaN   
5118 -0.476368  NaN       NaN     -0.476368       NaN               NaN   
5119 -0.484596  NaN       NaN     -0.484596       NaN               NaN   

      kurtosis  skewness  quantile_01  quantile_05  quantile_25  quantile_75  \
0          NaN     

# Calculate features per time window
features = []
for idx, df_window in df.groupby(df.index): # grouped by time window
    features_dict = {
        'mean': df_window['activity_norm'].mean(),
        'std': df_window['activity_norm'].std(),
        'variance': df_window['activity_norm'].var(),
        'variance': df_window['activity_norm'].var(),
        'trimmed_mean': df_window['activity_norm'].quantile(0.05),
        'coef_var': df_window['activity_norm'].std() / df_window['activity_norm'].mean(),
        'inverse_coef_var': df_window['activity_norm'].mean() / df_window['activity_norm'].std(),
        'kurtosis': df_window['activity_norm'].kurtosis(),
        'skewness': df_window['activity_norm'].skew(),
        'quantile_1': df_window['activity_norm'].quantile(0.01),
        'quantile_5': df_window['activity_norm'].quantile(0.05),
        'quantile_25': df_window['activity_norm'].quantile(0.25),
        'quantile_75': df_window['activity_norm'].quantile(0.75),
        'quantile_95': df_window['activity_norm'].quantile(0.95),
        'quantile_99': df_window['activity_norm'].quantile(0.99)
    }
    
    features.append(features_dict) 

features_df = pd.DataFrame(features)

In [59]:
print(features_hour.head(5))

                         mean  std  variance  quantile_01  quantile_05  \
timestamp                                                                
2002-10-02 15:00:00  0.030959  NaN       NaN     0.030959     0.030959   
2002-10-02 16:00:00  1.656566  NaN       NaN     1.656566     1.656566   
2002-10-02 17:00:00  2.274023  NaN       NaN     2.274023     2.274023   
2002-10-02 18:00:00  3.926951  NaN       NaN     3.926951     3.926951   
2002-10-02 19:00:00  1.126536  NaN       NaN     1.126536     1.126536   

                     quantile_25  quantile_75  quantile_95  quantile_99  \
timestamp                                                                 
2002-10-02 15:00:00     0.030959     0.030959     0.030959     0.030959   
2002-10-02 16:00:00     1.656566     1.656566     1.656566     1.656566   
2002-10-02 17:00:00     2.274023     2.274023     2.274023     2.274023   
2002-10-02 18:00:00     3.926951     3.926951     3.926951     3.926951   
2002-10-02 19:00:00     1.12653